In [ ]:
# !python -m pip install --upgrade pip

In [ ]:
# !pip install -r requirements.txt

In [ ]:
# !pip install matplotlib
# !pip install scikit-learn

In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import pickle
from torchvision.models import resnet50, vgg19
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def load_stanford_dogs(root_dir):
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    full_dataset = torchvision.datasets.ImageFolder(root=root_dir)
    
    train_size = int(0.6 * len(full_dataset))
    val_size = int(0.2 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])
    
    train_dataset.dataset.transform = train_transform
    val_dataset.dataset.transform = val_test_transform
    test_dataset.dataset.transform = val_test_transform
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
    
    return train_loader, val_loader, test_loader,train_dataset, val_dataset, test_dataset, len(full_dataset.classes)

In [4]:
train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset, num_classes = load_stanford_dogs('images/Images')
print(f"Number of classes: {num_classes}")

Number of classes: 120


In [5]:
from torchvision.models import resnet50

def get_res50_model(num_classes):
    model = resnet50(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [6]:
from torchvision.models import vgg19

def get_vgg19_model(num_classes):
    model = vgg19(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)
    
    return model

In [7]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# ## visualising the split (I wanted to be sure it was doing it correctly)
# def visualize_class_distribution(dataset, title):
#     labels = [label for _, label in dataset]
    
#     plt.figure(figsize=(12, 6))
#     sns.countplot(x=labels)
#     plt.xlabel('Class')
#     plt.ylabel('Count')
#     plt.title(f'Class Distribution - {title}')
#     plt.xticks(rotation=45)
#     plt.show()

# # visualize_class_distribution(train_dataset, 'Train Dataset')
# visualize_class_distribution(val_dataset, 'Validation Dataset')
# # visualize_class_distribution(test_dataset, 'Test Dataset')

In [8]:
def train_eval_model(model, model_name, train_loader, val_loader, test_loader, num_epochs=10, learning_rate=0.001, batch_size=32):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

    best_val_accuracy = 0.0
    best_model = None
    hyperparameter_record = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'{model_name} - Epoch {epoch+1}, Batch {i+1}, Loss: {running_loss/100:.3f}')
                running_loss = 0.0
        
        scheduler.step()
        
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_accuracy = 100 * val_correct / val_total
        print(f'{model_name} - Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy:.2f}%')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = model.state_dict()
        
        hyperparameter_record.append({
            'epoch': epoch+1,
            'learning_rate': learning_rate,
            'batch_size': batch_size,
            'validation_accuracy': val_accuracy
        })

    print(f'{model_name} - Training completed!')

    model.load_state_dict(best_model)
    model.eval()

    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

    test_accuracy = 100 * test_correct / test_total
    print(f'{model_name} - Final Test Accuracy: {test_accuracy:.2f}%')

    return model, hyperparameter_record

In [ ]:
# resnet50_model = get_res50_model(num_classes)
# vgg19_model = get_vgg19_model(num_classes)

# trained_resnet50 = train_eval_model(resnet50_model, 'ResNet50', train_loader, val_loader, test_loader)
# trained_vgg19 = train_eval_model(vgg19_model, 'VGG19', train_loader, val_loader, test_loader)

In [ ]:
# learning_rates = [0.001, 0.01, 0.1]
# batch_sizes = [32, 64, 128]

# for lr in learning_rates:
#     for bs in batch_sizes:
#         # ResNet50
#         resnet50_model = get_res50_model(num_classes)
#         trained_resnet50, resnet50_record = train_eval_model(resnet50_model, 'ResNet50', train_loader, val_loader, test_loader, learning_rate=lr, batch_size=bs)
#         print(f"ResNet50 - Learning Rate: {lr}, Batch Size: {bs}")
#         print(resnet50_record)
#         print("---")

In [ ]:
# for lr in learning_rates:
#     for bs in batch_sizes:      
#         # VGG19
#         vgg19_model = get_vgg19_model(num_classes)
#         trained_vgg19, vgg19_record = train_eval_model(vgg19_model, 'VGG19', train_loader, val_loader, test_loader, learning_rate=lr, batch_size=bs)
#         print(f"VGG19 - Learning Rate: {lr}, Batch Size: {bs}")
#         print(vgg19_record)
#         print("---")

In [9]:
resnet50_model = get_res50_model(num_classes)
trained_resnet50,_ = train_eval_model(resnet50_model, 'ResNet50', train_loader, val_loader, test_loader, learning_rate=0.001, batch_size=32)
print("---")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet50 - Epoch 1, Batch 100, Loss: 3.115
ResNet50 - Epoch 1, Batch 200, Loss: 1.309
ResNet50 - Epoch 1, Batch 300, Loss: 1.048
ResNet50 - Epoch 1/10, Validation Accuracy: 76.55%
ResNet50 - Epoch 2, Batch 100, Loss: 0.673
ResNet50 - Epoch 2, Batch 200, Loss: 0.688
ResNet50 - Epoch 2, Batch 300, Loss: 0.665
ResNet50 - Epoch 2/10, Validation Accuracy: 78.35%
ResNet50 - Epoch 3, Batch 100, Loss: 0.527
ResNet50 - Epoch 3, Batch 200, Loss: 0.569
ResNet50 - Epoch 3, Batch 300, Loss: 0.577
ResNet50 - Epoch 3/10, Validation Accuracy: 79.98%
ResNet50 - Epoch 4, Batch 100, Loss: 0.435
ResNet50 - Epoch 4, Batch 200, Loss: 0.464
ResNet50 - Epoch 4, Batch 300, Loss: 0.499
ResNet50 - Epoch 4/10, Validation Accuracy: 80.05%
ResNet50 - Epoch 5, Batch 100, Loss: 0.379
ResNet50 - Epoch 5, Batch 200, Loss: 0.383
ResNet50 - Epoch 5, Batch 300, Loss: 0.413
ResNet50 - Epoch 5/10, Validation Accuracy: 78.26%
ResNet50 - Epoch 6, Batch 100, Loss: 0.345
ResNet50 - Epoch 6, Batch 200, Loss: 0.353
ResNet50 - Epo

In [10]:
vgg19_model = get_vgg19_model(num_classes)
trained_vgg19,_ = train_eval_model(vgg19_model, 'VGG19', train_loader, val_loader, test_loader, learning_rate=0.001, batch_size=32)
print("---")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG19 - Epoch 1, Batch 100, Loss: 1.640
VGG19 - Epoch 1, Batch 200, Loss: 0.864
VGG19 - Epoch 1, Batch 300, Loss: 0.871
VGG19 - Epoch 1/10, Validation Accuracy: 79.74%
VGG19 - Epoch 2, Batch 100, Loss: 0.538
VGG19 - Epoch 2, Batch 200, Loss: 0.566
VGG19 - Epoch 2, Batch 300, Loss: 0.642
VGG19 - Epoch 2/10, Validation Accuracy: 80.98%
VGG19 - Epoch 3, Batch 100, Loss: 0.447
VGG19 - Epoch 3, Batch 200, Loss: 0.496
VGG19 - Epoch 3, Batch 300, Loss: 0.543
VGG19 - Epoch 3/10, Validation Accuracy: 79.96%
VGG19 - Epoch 4, Batch 100, Loss: 0.413
VGG19 - Epoch 4, Batch 200, Loss: 0.451
VGG19 - Epoch 4, Batch 300, Loss: 0.476
VGG19 - Epoch 4/10, Validation Accuracy: 80.66%
VGG19 - Epoch 5, Batch 100, Loss: 0.383
VGG19 - Epoch 5, Batch 200, Loss: 0.369
VGG19 - Epoch 5, Batch 300, Loss: 0.434
VGG19 - Epoch 5/10, Validation Accuracy: 80.49%
VGG19 - Epoch 6, Batch 100, Loss: 0.366
VGG19 - Epoch 6, Batch 200, Loss: 0.385
VGG19 - Epoch 6, Batch 300, Loss: 0.396
VGG19 - Epoch 6/10, Validation Accuracy:

In [ ]:
# # Save the trained model and important variables in /workspace/model
# torch.save(trained_resnet50.state_dict(), '/workspace/model/trained_resnet50.pth')
# torch.save(trained_vgg19.state_dict(), '/workspace/model/trained_vgg19.pth')
# variables = {
#     'train_loader': train_loader,
#     'val_loader': val_loader,
#     'test_loader': test_loader,
#     'train_dataset': train_dataset,
#     'val_dataset': val_dataset,
#     'test_dataset': test_dataset,
#     'num_classes': num_classes,
#     'resnet50_record': resnet50_record
# }

In [ ]:
# with open('/workspace/model/variables.pkl', 'rb') as f:
#     variables = pickle.load(f)

# train_loader = variables['train_loader']
# val_loader = variables['val_loader']
# test_loader = variables['test_loader']
# train_dataset = variables['train_dataset']
# val_dataset = variables['val_dataset']
# test_dataset = variables['test_dataset']
# num_classes = variables['num_classes']
# resnet50_record = variables['resnet50_record']

In [ ]:
# # Load models
# trained_resnet50 = resnet50(pretrained=False)
# trained_resnet50.fc = nn.Linear(trained_resnet50.fc.in_features, num_classes)
# trained_resnet50.load_state_dict(torch.load('/workspace/model/trained_resnet50.pth'))
# trained_resnet50.to(device)

# trained_vgg19 = vgg19(pretrained=False)
# trained_vgg19.classifier[-1] = nn.Linear(trained_vgg19.classifier[-1].in_features, num_classes)
# trained_vgg19.load_state_dict(torch.load('/workspace/model/trained_vgg19.pth'))
# trained_vgg19.to(device)

In [11]:
import gradio as gr

# Gradio interface
def predict(image):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        resnet50_output = trained_resnet50(image)
        vgg19_output = trained_vgg19(image)
    
    resnet50_prob = torch.nn.functional.softmax(resnet50_output, dim=1)
    vgg19_prob = torch.nn.functional.softmax(vgg19_output, dim=1)
    
    resnet50_pred = torch.argmax(resnet50_prob, dim=1).item()
    vgg19_pred = torch.argmax(vgg19_prob, dim=1).item()
    
    resnet50_class = train_dataset.dataset.classes[resnet50_pred]
    vgg19_class = train_dataset.dataset.classes[vgg19_pred]
    
    resnet50_class = resnet50_class.split('-')[-1].strip()
    vgg19_class = vgg19_class.split('-')[-1].strip()
    
    return resnet50_class, vgg19_class

with gr.Blocks() as demo:
    gr.Markdown("# Dog Breed Classification")
    gr.Markdown("Upload an image of a dog and get the predicted breed using ResNet50 and VGG19 models.")
    
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type='pil')
            greet_btn = gr.Button("Predict")
        
        with gr.Column():
            output_text1 = gr.Textbox(label="ResNet50 Prediction")
            output_text2 = gr.Textbox(label="VGG19 Prediction")
    
    greet_btn.click(fn=lambda image: predict(image), inputs=image_input, outputs=[output_text1, output_text2])

demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://115de1a4ca5d7de806.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# Dog Breed Classification Project Report

## Introduction
The goal of this project was to classify dog breeds using deep learning techniques. We utilized the Stanford Dogs Dataset, which contains images of 120 different dog breeds. The objective was to train models that can accurately predict the breed of a dog given an input image.

## Data Preparation
The Stanford Dogs Dataset was loaded using PyTorch's `ImageFolder` class. The dataset was split into training (60%), validation (20%), and test (20%) sets using the `random_split` function. Data preprocessing steps included resizing the images to (224, 224) pixels, applying random horizontal flips for data augmentation, converting the images to tensors, and normalizing the pixel values using pre-defined mean and standard deviation values.

## Model Architecture
We employed two pre-trained models for this project: ResNet50 and VGG19. These models were chosen due to their strong performance on image classification tasks. We utilized transfer learning by initializing the models with pre-trained weights from the ImageNet dataset. The final classification layer of each model was replaced with a new linear layer to match the number of dog breeds (120) in our dataset.

## Training Process
The models were trained using the cross-entropy loss function and the Adam optimizer with an initial learning rate of 0.001. A learning rate scheduler was used to decay the learning rate by a factor of 0.1 every 7 epochs. The training loop was run for 10 epochs, and the model with the best validation accuracy was saved for evaluation.

## Evaluation Results
After training, the models were evaluated on the test set. The ResNet50 model achieved a final test accuracy of 82.56%, while the VGG19 model achieved an accuracy of 82.19%. These results demonstrate the effectiveness of the trained models in accurately classifying dog breeds.

## Hyperparameter Tuning
To optimize the models' performance, we conducted hyperparameter tuning experiments. We varied the learning rate (0.001, 0.01, 0.1) and batch size (32, 64, 128) and observed their impact on validation accuracy. The results showed that a learning rate of 0.001 and a batch size of 32 yielded the best performance for both the ResNet50 and VGG19 models.

## Insights and Discussion
Both ResNet50 and VGG19 models performed well on the dog breed classification task, achieving similar test accuracies. Transfer learning proved to be effective in leveraging pre-trained weights and adapting them to our specific dataset.

One challenge faced during the project was the presence of visually similar dog breeds, which can be difficult to distinguish even for humans. Future improvements could include exploring more advanced data augmentation techniques, using ensemble methods, or incorporating additional information such as breed descriptions or characteristics.

## Conclusion
In this project, we successfully trained deep learning models to classify dog breeds using the Stanford Dogs Dataset. The ResNet50 and VGG19 models achieved high accuracies of 82.56% and 82.19%, respectively, demonstrating the effectiveness of transfer learning and hyperparameter tuning. This dog breed classification system can be used in various applications, such as pet identification or assisting in animal shelters.

The project highlights the importance of careful data preparation, model selection, and hyperparameter tuning in achieving optimal performance. It also showcases the power of deep learning in solving real-world image classification problems.

## References
- Stanford Dogs Dataset: http://vision.stanford.edu/aditya86/ImageNetDogs/
- PyTorch Documentation: https://pytorch.org/docs/stable/index.html
- Transfer Learning Tutorial: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html